# Stacked-ensemble classifiers (in-class exercise)

The idea is to build a stacked ensemble to make accurate predictions about tumor. 

The stacked ensemble works as follows

- Level 0: the level zero classifiers predict, after training, the input data and generate the predicted output data.
- Level 1: the level one classifier uses the predictions of the level zero classifiers as inputs and predict the output. This output is then compared to the true label to check accuracies and other performance metrics.


For this exercise we will use the following classifiers:
1. Support Vector Machine (SVM)
2. Linear Discriminant Analysis (LDA)
3. Quadratic Discriminant Analysis (QDA)
4. Random Forest (RF)


## 1. Loading the dataset
Let us begin with loading and preprocessing the dataset.

Here LabelEncoder() is very useful as it converts the classes (denoted here with the type of tumor (I guess)) from strings to integers.

In [1]:
import pandas as pd
import numpy as np

# read the dataset
dt = pd.read_csv('hgsc.csv')
dt.head()

,class,ABAT,ABHD2,ACTB,ACTR2,ACTR5,ACVR2A,ADAMDEC1,ADCYAP1R1,AEBP1,...,WT1,XPO7,XPOT,YTHDC2,ZDHHC14,ZDHHC7,ZEB1,ZFP36,ZHX3,ZNF423
0,PRO.C5,-0.010674,0.263376,-0.115492,-0.323565,0.005161,-0.504271,-1.283720,-0.433908,0.673072,...,0.077048,0.459961,-0.072049,0.243935,-0.056318,-0.204971,0.179639,-0.292136,-0.034261,0.490152
1,MES.C1,-0.710741,0.110421,0.532555,-0.253877,-0.389024,-0.121941,-1.732920,-0.727880,1.706110,...,0.547120,-0.674773,-0.236746,0.551354,0.215982,0.196677,1.467320,2.461040,0.415041,2.116880
2,DIF.C4,0.881506,0.372862,0.052344,0.028721,-0.848119,-1.281180,1.524370,-0.288317,-2.010830,...,1.058170,0.350895,-0.000051,0.010498,0.592285,-0.338954,-0.842242,0.096242,-0.471005,-1.662190
3,MES.C1,-1.085090,0.415651,0.395376,-0.271050,0.146536,-0.363270,0.993823,-0.450427,1.999170,...,-0.677226,-0.109778,0.033163,0.760080,-1.169030,0.325604,1.785760,-0.212328,0.537493,-0.102138
4,MES.C1,-0.932230,0.045352,0.595068,0.187856,-0.200287,0.211144,1.844640,-0.416482,1.327800,...,0.961688,-0.009010,0.529045,-0.551470,-0.188697,0.157393,0.469166,1.748000,0.144196,-0.561641


In [2]:
dt=dt.dropna()

In [3]:
# Now x, y
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

X = dt.loc[:,dt.columns!='class']
#dt['class']=dt['class'].astype('integer') # the following is useless.. We can use LabelEncoder from sklearn.preprocessing
y = dt['class']
y = le.fit_transform(y)

In [4]:
# checking the shape of the predictors.
X.shape

(489, 321)

In [5]:
# now split in test and train set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5, random_state=42)

In [6]:
# Checking the PCA.. it might be useful for LDA and QDA (their accuracy is terrible..)
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train_std=sc.fit_transform(X_train)
pca = PCA(n_components = None)
pca.fit(X_train_std)
pca.explained_variance_ratio_

array([  2.01982714e-01,   9.80500683e-02,   4.50952245e-02,
         3.28677337e-02,   2.71819534e-02,   2.63146339e-02,
         2.32510173e-02,   2.04708606e-02,   1.78945311e-02,
         1.70066134e-02,   1.60041367e-02,   1.41087792e-02,
         1.37462630e-02,   1.27609644e-02,   1.17572668e-02,
         1.15436857e-02,   1.10070860e-02,   1.04476933e-02,
         9.93750956e-03,   9.75405495e-03,   9.26826009e-03,
         8.97313656e-03,   8.35131226e-03,   8.31322249e-03,
         8.15083584e-03,   7.74742475e-03,   7.48789495e-03,
         7.17772807e-03,   7.00037755e-03,   6.83781926e-03,
         6.48984306e-03,   6.30554819e-03,   5.97381484e-03,
         5.85819461e-03,   5.67160016e-03,   5.45704134e-03,
         5.35321359e-03,   5.21310330e-03,   5.07535720e-03,
         4.97845240e-03,   4.66597872e-03,   4.65741960e-03,
         4.54072631e-03,   4.52548451e-03,   4.43649612e-03,
         4.33814835e-03,   4.20124994e-03,   4.12575416e-03,
         4.11653871e-03,

## 2. Tuning the single classifiers

### 2.1 Tuning a SVM 

In [35]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV


pipe_svc = Pipeline([('scl', StandardScaler()),
            ('clf', SVC(random_state=1, probability=True))])


# This stuff is to choose the best paramters of the 
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

param_grid = [{'clf__C': param_range, 
               'clf__kernel': ['linear']},
                 {'clf__C': param_range, 
                  'clf__gamma': param_range, 
                  'clf__kernel': ['rbf']}]

gs = GridSearchCV(estimator=pipe_svc, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  cv=10,
                  n_jobs=-1)
gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)


0.8934426229508197
{'clf__C': 0.01, 'clf__kernel': 'linear'}


Cool, so the best SVM for this problem seems to be a linear svm with C=0.01. So we'll use that in our stacked ensemble.

In [8]:
# 
gs.best_estimator_.fit(X_train, y_train)
print(gs.best_estimator_.score(X_test, y_test))


0.865306122449


In [25]:
from sklearn.metrics import confusion_matrix

y_pred_best = gs.best_estimator_.predict(X_test)

confmat = confusion_matrix(y_true = y_test, y_pred=y_pred_best)
print('SVM confmat:\n',confmat)

SVM confmat:
 [[67  2  3  2]
 [ 4 45  2  0]
 [ 4  2 48  1]
 [ 6  4  3 52]]


### 2.2. Linear Discriminant Analysis
Moving with the Linear Discriminant Analysis

In [24]:
# Now define a LDA pipeline.
#from sklearn.lda import LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

pipe_lda = Pipeline([('sc', StandardScaler()),
                    ('clf', LDA())])

lda = LDA()

pipe_lda.fit(X_train, y_train)
lda.fit(X_train, y_train)

print('pipe_lda_accuracy:',pipe_lda.score(X_test, y_test))
print('lda_accuracy:',lda.score(X_test, y_test))

y_pred_pipe = pipe_lda.predict(X_test)
confmat=  confusion_matrix(y_true = y_test, y_pred=y_pred_pipe)
print('pipe lda confmat:\n',confmat)

y_pred = lda.predict(X_test)
confmat=  confusion_matrix(y_true = y_test, y_pred=y_pred)
print('lda confmat: \n',confmat)


pipe_lda_accuracy: 0.563265306122
lda_accuracy: 0.563265306122
pipe lda confmat:
 [[45  9 14  6]
 [15 21  8  7]
 [10  4 38  3]
 [15 11  5 34]]
lda confmat: 
 [[45  9 14  6]
 [15 21  8  7]
 [10  4 38  3]
 [15 11  5 34]]


/Users/alexzucca/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


The linear discriminant analysis seems a very weak classifier.

### 2.3 Quadratic Discriminant Analysis

In [26]:
# Next define the QDA pipeline
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA

pipe_qda = Pipeline([('sc', StandardScaler()),
                    ('clf', QDA())])
qda = QDA()

pipe_qda.fit(X_train, y_train)
qda.fit(X_train, y_train)

print(pipe_qda.score(X_test, y_test))
print(qda.score(X_test, y_test))

/Users/alexzucca/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


0.338775510204
0.387755102041


In [27]:
# I definitely choose the non standardize case. Now computing the confusion matrix
y_pred = qda.predict(X_test)
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print('QDA confmat: \n', confmat)


QDA confmat: 
 [[20 26 11 17]
 [10 23  9  9]
 [15 17 16  7]
 [11 11  7 36]]


The QDA is even weaker than the LDA. Not sure why..

### 2.4 Random Forest
Here I should tune up the parameters of the random forest classifier. 

In [28]:
# Last, let's define the Random Forest Pipeline

from  sklearn.ensemble import RandomForestClassifier 

pipe_rf = Pipeline([('sc', StandardScaler()),
                    ('clf', RandomForestClassifier(criterion='gini',random_state=1,n_jobs=4))])

rf = RandomForestClassifier(criterion='gini', 
                                random_state=1,
                                n_jobs=4)

# Actually I want to understand what's going on with this thing, so let's try the following.
rf.fit(X_train, y_train)
rf_acc = rf.score(X_test, y_test)
print('Random Forest accuracy:',rf_acc)

pipe_rf.fit(X_train,y_train)
pipe_rf_acc = pipe_rf.score(X_test,y_test)
print('Pipe Random Forest Accuracy', pipe_rf_acc)

                    

Random Forest accuracy: 0.840816326531
Pipe Random Forest Accuracy 0.840816326531


In [29]:
# no difference, let's keep the random forest with no normalization
y_pred = rf.predict(X_test)
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print('RF confmat:\n', confmat )

RF confmat:
 [[55  3  6 10]
 [ 2 46  3  0]
 [ 3  2 49  1]
 [ 5  2  2 56]]


## 3. Stacked Ensemble.
### 3.1. First trial: there is no J-folding here
The following code suffers (for sure) of overfitting. But it's a good starting point.

In [31]:
# Trying to code the stacked ensemble classifier

from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.externals import six
from sklearn.base import clone
from sklearn.pipeline import _name_estimators
from sklearn.model_selection import KFold
import numpy as np
import operator

class StackedEnsembleClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, lev0_clfs, lev1_clf, J ,weights = None):
        self.lev0_clfs = lev0_clfs
        self.named_clfs = {key: value for key, value in _name_estimators(lev0_clfs)}
        self.lev1_clf = lev1_clf
        self.weights = weights
        # number of folds
        self.J = J


    def fit(self, X, y):
                
        # lev0_clfs_ is the set of fitted classifiers.
        self.lev0_clfs_ = []
        
        # Now fit each classifier
        for clf in self.lev0_clfs:
            fitted_clf = clf.fit(X,y)
            self.lev0_clfs_.append(fitted_clf)
        
        # Initialize the array for the level 1 classifier
        rows = X.shape[0]
        columns = len(np.unique(y))*len(self.lev0_clfs)
        
        self.num_classes = len(np.unique(y))
        
        X2=np.zeros((rows, columns))
        
        # Prepare the data for the level 1 classifier
        for i in range(len(self.lev0_clfs_)):
            clf = self.lev0_clfs_[i]
            X_temp=clf.predict_proba(X)
            for j in range(X_temp.shape[1]):
                for k in range(X2.shape[0]):
                    c = len(np.unique(y))*i+j
                    X2[k][c] = X_temp[k][j]
        
        
        # now train the level 1 classifier
        self.lev1_clf_ = self.lev1_clf.fit(X2,y)
        
        # That's it        
        return self
    
    def predict(self,X):
        
        rows = X.shape[0]
        columns = self.num_classes*len(self.lev0_clfs_)
    
        X2=np.zeros((rows, columns))
        print('X2.shape:',X2.shape)
        
        for i in range(len(self.lev0_clfs_)):
            clf = self.lev0_clfs_[i]
            X_temp=clf.predict_proba(X)
            for j in range(X_temp.shape[1]):
                for k in range(X2.shape[0]):
                    c = len(np.unique(y))*i+j
                    #print(k,c)
                    X2[k][c] = X_temp[k][j]
                    
                    
        # now feed this stuff into the lev1_classifier
        y_pred = self.lev1_clf_.predict(X2)
        return(y_pred)
        
    def predict_proba(self,X):
        # will modify this later.
        
        rows = X.shape[0]
        columns = self.num_classes*len(self.lev0_clfs_)
    
        X2=np.zeros((rows, columns))
        print('X2.shape:',X2.shape)
        
        for i in range(len(self.lev0_clfs_)):
            clf = self.lev0_clfs_[i]
            X_temp=clf.predict_proba(X)
            for j in range(X_temp.shape[1]):
                for k in range(X2.shape[0]):
                    c = len(np.unique(y))*i+j
                    #print(k,c)
                    X2[k][c] = X_temp[k][j]
                    
                    
        # now feed this stuff into the lev1_classifier
        
        p_pred = self.lev1_clf_.predict_proba(X)
        
        return(p_pred)
    
        # return prediction probabilities

In [36]:
from sklearn.linear_model import LogisticRegression
# Now combining the stuff..

# this is going to be the level-1 classifier
lr = LogisticRegression()

# these are the level-0 classifiers.
#getting the best estimator from sect. 2.1
svc = gs.best_estimator_
rf = RandomForestClassifier()
lda = LDA()
qda=QDA()


sec=StackedEnsembleClassifier([svc,rf,lda,qda], lr, J=2)

In [37]:
# Now try to fit the thing
sc = StandardScaler()

X_train_std = sc.fit_transform(X_train)

In [38]:
# Now train the ensemble.
sec.fit(X_train, y_train)

/Users/alexzucca/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/alexzucca/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


StackedEnsembleClassifier(J=2,
             lev0_clfs=[Pipeline(memory=None,
     steps=[('scl', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=1,...ors=None, reg_param=0.0,
               store_covariance=False, store_covariances=None, tol=0.0001)],
             lev1_clf=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
             weights=None)

In [39]:
y_pred = sec.predict(X_test)

X2.shape: (245, 16)


In [40]:
accuracy = 1.0 - (y_pred != y_test).sum()/len(y_test)

In [41]:
print('accuracy Stacked Ensemble:', accuracy)

accuracy Stacked Ensemble: 0.771428571429


In [42]:
# Now I want to compute the confusion matrix here
confmat = confusion_matrix(y_true = y_test, y_pred=y_pred)

In [43]:
print('Stacked Ensemble confmat:\n', confmat)

Stacked Ensemble confmat:
 [[55 10  3  6]
 [ 7 37  5  2]
 [ 5  4 44  2]
 [ 6  4  2 53]]


In [49]:
# The issue might come from the QDA. Let's avoid it
sec=StackedEnsembleClassifier([svc,rf, lda], lr, J=2)
sec.fit(X_train, y_train)
y_pred = sec.predict(X_test)
accuracy = 1.0 - (y_pred != y_test).sum()/len(y_test)
print('accuracy Stacked Ensemble:', accuracy)
confmat = confusion_matrix(y_true = y_test, y_pred=y_pred)
print('Stacked Ensemble confmat:\n', confmat)

X2.shape: (245, 12)
accuracy Stacked Ensemble: 0.767346938776
Stacked Ensemble confmat:
 [[55  7  7  5]
 [ 9 35  7  0]
 [ 4  2 47  2]
 [ 7  4  3 51]]


/Users/alexzucca/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


It looks like we're not good enough. Let's see what happens if we get rid of the lda.

In [50]:
# The issue might come from the QDA. Let's avoid it
sec=StackedEnsembleClassifier([svc,rf], lr, J=2)
sec.fit(X_train, y_train)
y_pred = sec.predict(X_test)
accuracy = 1.0 - (y_pred != y_test).sum()/len(y_test)
print('accuracy Stacked Ensemble:', accuracy)
confmat = confusion_matrix(y_true = y_test, y_pred=y_pred)
print('Stacked Ensemble confmat:\n', confmat)

X2.shape: (245, 8)
accuracy Stacked Ensemble: 0.902040816327
Stacked Ensemble confmat:
 [[67  0  3  4]
 [ 1 48  2  0]
 [ 5  1 49  0]
 [ 4  2  2 57]]


It works better. Good

### 3.2. Implementing J-foldings here

In [ ]:
# Trying to code the stacked ensemble classifier

# Before implementing the actual J-fold methid, I want to train and predict on the same thing... just to practice.

# Now J-folding

from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.externals import six
from sklearn.base import clone
from sklearn.pipeline import _name_estimators
from sklearn.model_selection import KFold
import numpy as np
import operator

class StackedEnsembleClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, lev0_clfs, lev1_clf, J ,weights = None):
        self.lev0_clfs = lev0_clfs
        self.named_clfs = {key: value for key, value in _name_estimators(lev0_clfs)}
        self.lev1_clf = lev1_clf
        self.weights = weights
        # number of folds
        self.J = J


    def fit(self, X, y):
        
        # Initialize the k-fold
        
        # Initialize the Kfold stuff
        #kf = KFold(n_splits = self.J, random_state=42)
        
        # lev0_clfs_ is the set of fitted classifiers.
        self.lev0_clfs_ = []
        
        # Now fit each classifier
        for clf in self.lev0_clfs:
            fitted_clf = clf.fit(X,y)
            self.lev0_clfs_.append(fitted_clf)
            
        #print('classifiers fitted..')
        
        
        # Initialize the array for the level 1 classifier
        rows = X.shape[0]
        columns = len(np.unique(y))*len(self.lev0_clfs)
        
        self.num_classes = len(np.unique(y))
        
        X2=np.zeros((rows, columns))
        #print('X2.shape:',X2.shape)


        #first_iter = True
        #ind=0
        #for (train, test) in kf.split(X, y):
        #    #print('k:',k)
        #    ind+=1
        #    print('ind:', ind)

            # Initialize a temporary array
        #    rows_temp = len(y[test])
        #    X_temp = np.zeros((rows_temp, columns))
        #    
            # level 0 for the k-th fold
        #    for i in range(len(self.lev0_clfs)):
        #        self.lev0_clfs[i].fit(X[train], y[train])
        #        X_temp2 = self.lev0_clfs[i].predict_proba(X[test])
                
        #        for n in range(X_temp.shape[0]):
        #            for m in range(X_temp2.shape[1]):
        #                c = len(np.unique(y))*i+m
        #                X_temp[n][c] = X_temp2[n][m]
        #                
        #    if (first_iter):
        #        X2 = X_temp
        #        first_iter = False
        #    else:
        #        X2 = np.concatenate((X2,X_temp))
              
            
        #print('X2.shape:', X2.shape)
                
                        
        
        
        for i in range(len(self.lev0_clfs_)):
            clf = self.lev0_clfs_[i]
            X_temp=clf.predict_proba(X)
            for j in range(X_temp.shape[1]):
                for k in range(X2.shape[0]):
                    c = len(np.unique(y))*i+j
        #            #print(k,c)
                    X2[k][c] = X_temp[k][j]
        
        
        # now train the level 1 classifier
        self.lev1_clf_ = self.lev1_clf.fit(X2,y)
        
        # That's it        
        return self
    
    def predict(self,X):
        
        rows = X.shape[0]
        columns = self.num_classes*len(self.lev0_clfs_)
    
        X2=np.zeros((rows, columns))
        print('X2.shape:',X2.shape)
        
        for i in range(len(self.lev0_clfs_)):
            clf = self.lev0_clfs_[i]
            X_temp=clf.predict_proba(X)
            for j in range(X_temp.shape[1]):
                for k in range(X2.shape[0]):
                    c = len(np.unique(y))*i+j
                    #print(k,c)
                    X2[k][c] = X_temp[k][j]
                    
                    
        # now feed this stuff into the lev1_classifier
        y_pred = self.lev1_clf_.predict(X2)
        return(y_pred)
        # return prediction
        
    def predict_proba(self,X):
        # will modify this later.
        
        rows = X.shape[0]
        columns = self.num_classes*len(self.lev0_clfs_)
    
        X2=np.zeros((rows, columns))
        print('X2.shape:',X2.shape)
        
        for i in range(len(self.lev0_clfs_)):
            clf = self.lev0_clfs_[i]
            X_temp=clf.predict_proba(X)
            for j in range(X_temp.shape[1]):
                for k in range(X2.shape[0]):
                    c = len(np.unique(y))*i+j
                    #print(k,c)
                    X2[k][c] = X_temp[k][j]
                    
                    
        # now feed this stuff into the lev1_classifier
        
        p_pred = self.lev1_clf_.predict_proba(X)
        
        return(p_pred)
    
        # return prediction probabilities